<a href="https://colab.research.google.com/github/ykitaguchi77/CorneAI/blob/main/GradCAM_Revision_250718.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **【目次】論文リビジョン用 CorneAI 分析ノートブック**

### **はじめに**
このノートブックは、角膜画像データセットに対するYOLOv5モデルの性能評価および、その判断根拠を複数の説明可能性AI（XAI）手法を用いて分析・評価することを目的とします。各セクションは独立して実行、または順次実行できるように構成されています。

---

### **第0部：環境構築と初期設定**
全ての分析の基礎となる環境をここで一括して設定します。

* **0.1. Google Driveのマウント**
* **0.2. 必須ライブラリのインストール**
* **0.3. リポジトリのクローンとパス設定**
* **0.4. 主要パラメータとパスの一元管理**
* **0.5. 共通関数の定義**
* **0.6. YOLOv5モデルのロード**

---

### **第1部：基礎データの準備とAIによる初期予測**
分析の土台となるCSVファイルを作成し、ベースラインとなるAIの予測結果を記録します。

* **1.1. 初期CSVファイルの作成**
* **1.2. ファイル名からのGround Truth抽出**
* **1.3. 全画像に対するYOLOv5推論の実行**
* **1.4. 予測結果と信頼度をCSVに追記**
* **1.5. 結果CSVの保存と確認**

---

### **第2部：説明可能性AI（XAI）分析とマスク画像の生成**
各XAI手法を実行し、モデルの判断根拠を可視化した「マスク画像」を生成・保存します。

* **2.1. GradCAM++による分析**
* **2.2. GradCAMによる分析**
* **2.3. LIMEによる分析**
* **2.4. Occlusion Sensitivityによる分析**
* **2.5. (オプション) RISE, SHAPによる分析**

---

### **第3部：評価指標の計算と記録**
生成したマスク画像と専門家の手動アノテーション（Expert Mask）を比較し、定量的評価を行います。

* **3.1. Pointing Game Accuracyの計算**
* **3.2. Intersection over Union (IoU) の計算**
* **3.3. 最終評価CSVの保存**

---

### **第4部：Cut & Paste実験**
角膜領域を別の画像に移植し、モデルの頑健性を評価する追加実験です。

* **4.1. 実験用CSVの作成 (`Ueno_Mix1039_cutmix.csv`)**
* **4.2. Cut & Paste画像の生成と推論**
* **4.3. Cut & Paste予測結果の記録**

---

### **第5部：追加分析とデータ整理**
論文に必要なその他の補足データを算出します。

* **5.1. AI検出BBoxと専門家マスクの一致率計算 (`expert_ratio`)**
* **5.2. 最終統計分析と可視化**

---

##第0部：環境構築と初期設定

全ての分析の基礎となる環境をここで一括して設定します。このセクションのセルを上から順に実行すれば、以降の分析の準備がすべて整います。

0.1. Google Driveのマウント

Google Driveに接続します。

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

0.2. & 0.3. ライブラリのインストールと作業ディレクトリへの移動

分析に必要なライブラリをインストールし、yolov5-gradcamリポジトリをクローンして、そのディレクトリを基準に作業を進めます。

In [ ]:
# 必要なライブラリをインストール
%cd /content
!pip install -U git+https://github.com/pooya-mohammadi/deep_utils.git --q
!pip install japanize-matplotlib --q
!pip install scikit-image --q

# リポジトリをクローンして作業ディレクトリに設定
!git clone https://github.com/pooya-mohammadi/yolov5-gradcam
%cd /content/yolov5-gradcam

print("✅ 環境構築完了")

0.4. ライブラリの一括インポート

ノートブック全体で使用するライブラリをここでまとめてインポートします。

In [ ]:
# 基本ライブラリ
import os
import re
import warnings
from pathlib import Path

# データ操作・計算
import pandas as pd
import numpy as np
import cv2

# PyTorch関連
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F

# 可視化
import matplotlib.pyplot as plt
import japanize_matplotlib

# ユーティリティ
from tqdm.notebook import tqdm
from skimage.segmentation import quickshift

# YOLOv5 / Grad-CAM 関連
from models.experimental import attempt_load
from models.yolo import Model, Detect
from models.common import Conv, C3, SPPF
from utils.datasets import letterbox
from utils.general import non_max_suppression as yolo_nms

# 警告を抑制
warnings.filterwarnings("ignore", category=FutureWarning)

print("✅ ライブラリのインポート完了")

0.5. 主要パラメータとパスの一元管理

全てのファイルパスや設定値をこのセルで管理します。変更が必要な場合はこのセルのみを修正してください。

In [ ]:
# --- 主要パス設定 ---
BASE_DIR = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/Ueno_Mix1039"
IMAGE_DIR = os.path.join(BASE_DIR, "Images")
MODEL_PATH = "/gdrive/MyDrive/Deep_learning/CorneAI_nagoya/yolo5_forcresco/weights/eye_nii_2202_onecaseoneimage2_doctorcompare_yolov5s_epoch200_batch16_89.8p/last.pt"

# --- CSV関連パス ---
# このCSVが全ての分析結果を集約するマスターファイルとなります
MAIN_CSV_PATH = os.path.join(BASE_DIR, "Ueno_Mix1039_compare_methods.csv")
CUTMIX_CSV_PATH = os.path.join(BASE_DIR, "Ueno_Mix1039_cutmix.csv")

# --- マスク(アノテーション)関連パス ---
EXPERT_MASK_DIR = os.path.join(BASE_DIR, "Expert_annotation_masks")
XML_ANNOTATION_PATH = os.path.join(BASE_DIR, "Cornea_segmentation/annotations.xml")

# --- 分析結果の保存先ディレクトリ ---
BBOX_MASK_DIR = os.path.join(BASE_DIR, "YOLO_bbox_mask")
GRADCAM_PP_MASK_DIR = os.path.join(BASE_DIR, "AOI_50_mask_GradCAM++")
GRADCAM_MASK_DIR = os.path.join(BASE_DIR, "AOI_50_mask_GradCAM")
LIME_MASK_DIR = os.path.join(BASE_DIR, "AOI_50_mask_LIME")
OCCLUSION_MASK_DIR = os.path.join(BASE_DIR, "AOI_50_mask_Occlusion_sensitivity")

# --- モデル設定 ---
DEVICE = "cpu"  # GPUの個体差をなくし結果を統一するためCPUを使用
CLASS_NAMES = ["infection", "normal", "non-infection", "scar", "tumor", "deposit", "APAC", "lens opacity", "bullous"]
CONF_THRES = 0.25
IOU_THRES = 0.45

print("✅ パラメータ設定完了")

0.6. 共通関数の定義

複数のタスクで繰り返し使用する関数をここで定義します。

In [ ]:
def preprocess_image(img_bgr, device, img_size=(640, 640)):
    """画像をYOLOv5の入力形式に前処理する"""
    # BGR to RGB, to 640x640 with letterbox
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    img_resized = letterbox(img_rgb, new_shape=img_size, auto=True)[0]

    # HWC to CHW, to float tensor
    img_transposed = np.ascontiguousarray(img_resized.transpose((2, 0, 1)))
    img_tensor = torch.from_numpy(img_transposed).to(device).float() / 255.0

    # Add batch dimension
    if img_tensor.ndim == 3:
        img_tensor = img_tensor[None]

    return img_tensor, img_resized

print("✅ 共通関数の定義完了")

0.7. YOLOv5モデルのロード

分析の根幹となるYOLOv5モデルをロードします。ノートブック全体でこのmodel変数を使い回すことで、何度もロードする無駄をなくします。

In [ ]:
print(f"'{MODEL_PATH}' からモデルをロードしています...")
print(f"使用デバイス: {DEVICE}")

# PyTorchのカスタムクラスを安全にロードするための設定
# エラー回避のために必要
torch.serialization.add_safe_globals([
    Model, Detect, Conv, C3, SPPF, nn.Sequential, nn.ModuleList,
    nn.Conv2d, nn.BatchNorm2d, nn.SiLU, nn.MaxPool2d, nn.Upsample
])

model = attempt_load(MODEL_PATH, device=DEVICE)
model.eval()

print("\n✅ モデルのロード完了")
print(f"モデルクラス: {model.names}")

## 第1部：基礎データの準備とAIによる初期予測

分析の土台となるCSVファイルを作成し、ベースラインとなるAIの予測結果を記録します。

1.1. & 1.2. 初期CSVファイルの作成とGround Truthの抽出

画像フォルダをスキャンし、ファイル名を基にimage_basenameとGroundTruth列を持つDataFrameを作成します。

In [ ]:
def extract_ground_truth(basename):
    """ファイル名から[]内の文字列を抽出し、対応するクラス名を返す"""
    match = re.search(r"\[([^\]]+)\]", basename)
    if not match:
        return None

    label_text = match.group(1).lower()

    # ラベルとクラス名のマッピング辞書
    label_mapping = {
        'infection': 'infection',
        'normal': 'normal',
        'immun': 'non-infection',
        'scar': 'scar',
        'tumor': 'tumor',
        'deposit': 'deposit',
        'apac': 'APAC',
        'cat': 'lens opacity',
        'bullous': 'bullous'
    }

    for key, value in label_mapping.items():
        if key in label_text:
            return value

    return None

# 画像ディレクトリからファイル名リストを取得
image_extensions = {'.jpg', '.jpeg', '.png'}
image_files = [f for f in os.listdir(IMAGE_DIR) if Path(f).suffix.lower() in image_extensions]
basenames = [Path(f).stem for f in image_files]

# DataFrameを作成
df = pd.DataFrame({'image_basename': basenames})

# GroundTruth列を追加
df['GroundTruth'] = df['image_basename'].apply(extract_ground_truth)

print(f"✅ {len(df)}件の画像からベースとなるDataFrameを作成しました。")
display(df.head())

1.3. & 1.4. 全画像に対するYOLOv5推論と結果の追記

作成したリストの各画像に対して推論を実行し、予測結果（Predict）と信頼度（Likelihood）をDataFrameに追加します。

In [ ]:
predictions = []
confidences = []

# 第0部でロード済みのモデルを使用して推論
for basename in tqdm(df['image_basename'], desc="AIによる予測を実行中"):
    # 対応する画像ファイルのパスを検索
    image_file_found = None
    for ext in image_extensions:
        p = Path(IMAGE_DIR) / f"{basename}{ext}"
        if p.exists():
            image_file_found = p
            break

    if not image_file_found:
        predictions.append(None)
        confidences.append(None)
        continue

    img = cv2.imread(str(image_file_found))
    img_tensor, _ = preprocess_image(img, device=DEVICE)

    with torch.no_grad():
        pred = model(img_tensor)[0]
        pred_nms = yolo_nms(pred, conf_thres=CONF_THRES, iou_thres=IOU_THRES)[0]

        if pred_nms is not None and len(pred_nms) > 0:
            # 最も信頼度の高い予測を採用
            best_pred = pred_nms[0]
            pred_class_idx = int(best_pred[5])
            predictions.append(CLASS_NAMES[pred_class_idx])
            confidences.append(best_pred[4].item())
        else:
            # 何も検出されなかった場合
            predictions.append("none")
            confidences.append(0.0)

df['Predict'] = predictions
df['Likelihood'] = confidences

print("✅ 全画像の予測が完了しました。")

1.5. 結果CSVの保存と確認

全ての情報をまとめたDataFrameを、マスターCSVファイルとして保存します。

In [ ]:
# 分析手法ごとの列をあらかじめ作成（この後のタスクで使用）
xai_methods = ['GradCAM', 'GradCAM++', 'RISE', 'LIME', 'Occlusion']
for method in xai_methods:
    df[f'{method}_X'] = pd.NA
    df[f'{method}_Y'] = pd.NA
    df[f'{method}_pointing_game'] = pd.NA
    df[f'{method}_IoU'] = pd.NA

# CSVファイルとして保存
df.to_csv(MAIN_CSV_PATH, index=False, encoding='utf-8-sig')

print(f"✅ 全ての初期データをCSVファイルに保存しました: {MAIN_CSV_PATH}")

# 保存した内容の確認
print("\n--- 保存されたデータの先頭5行 ---")
display(df.head())

print("\n--- データ概要 ---")
df.info()

print("\n--- AIの予測結果の内訳 ---")
print(df['Predict'].value_counts())

## 第2部：説明可能性AI（XAI）分析とマスク画像の生成


2.1. GradCAM++による分析

YOLOv5モデルの指定された6つの畳み込み層に対してGradCAM++を適用します。「元画像と重ね合わせたヒートマップ」と「定量的評価用のAOIマスク」の両方を、それぞれ専用のフォルダに保存します。



In [ ]:
# --- ヒートマップを画像に重ね合わせるヘルパー関数 ---
def apply_heatmap_to_image(original_img, heatmap_normalized, colormap=cv2.COLORMAP_JET):
    """正規化されたヒートマップを元の画像に重ね合わせる"""
    heatmap_uint8 = np.uint8(255 * heatmap_normalized)
    heatmap_colored = cv2.applyColorMap(heatmap_uint8, colormap)
    overlaid_image = cv2.addWeighted(original_img, 0.6, heatmap_colored, 0.4, 0)
    return overlaid_image

# --- メイン処理 ---
df = pd.read_csv(MAIN_CSV_PATH)

target_layers = [
    "model_17_cv3_conv", "model_20_cv3_conv", "model_23_cv3_conv",
    "model_24_m_0", "model_24_m_1", "model_24_m_2"
]

# === 拡張子対応の修正 ===
# 事前に画像ファイルの「ベース名 -> フルパス」の辞書を作成
VALID_EXTENSIONS = {'.jpg', '.jpeg', '.png'}
path_map = {
    Path(f).stem: Path(IMAGE_DIR) / f
    for f in os.listdir(IMAGE_DIR)
    if Path(f).suffix.lower() in VALID_EXTENSIONS
}
print(f"✅ {len(path_map)}件の画像ファイル（.JPG, .Jpgなども含む）をインデックスしました。")

# --- 出力ディレクトリの準備 ---
base_output_dir = Path(GRADCAM_PP_MASK_DIR)
heatmap_base_dir = base_output_dir / "heatmaps"
mask_base_dir = base_output_dir / "masks"
for layer_name in target_layers:
    (heatmap_base_dir / layer_name).mkdir(parents=True, exist_ok=True)
    (mask_base_dir / layer_name).mkdir(parents=True, exist_ok=True)

print(f"GradCAM++分析を開始します。保存先: {base_output_dir}")

# 画像ごとにループ
for index, row in tqdm(df.iterrows(), total=len(df), desc="GradCAM++ 生成中"):
    basename = row['image_basename']

    # 辞書から画像パスを高速に取得
    img_path_found = path_map.get(basename)
    if not img_path_found:
        continue

    # 1つのレイヤーでも処理済みか簡易チェック（高速化のため）
    # 厳密なチェックはレイヤーごとのループ内で行う
    first_layer_mask_path = mask_base_dir / target_layers[0] / f"{basename}.png"
    if first_layer_mask_path.exists():
        continue

    img = cv2.imread(str(img_path_found))
    img_tensor, _ = preprocess_image(img, device=DEVICE)

    # 各レイヤーに対してループ
    for layer_name in target_layers:
        heatmap_path = heatmap_base_dir / layer_name / f"{basename}.jpg"
        mask_path = mask_base_dir / layer_name / f"{basename}.png"

        if heatmap_path.exists() and mask_path.exists():
            continue

        try:
            # methodを"gradcampp"に指定
            with YOLOV5GradCAM(model, layer_name, method="gradcampp") as cam:
                heatmap = cam(img_tensor)

            if heatmap is not None:
                original_h, original_w = img.shape[:2]
                heatmap_resized = cv2.resize(heatmap, (original_w, original_h), interpolation=cv2.INTER_LINEAR)

                # 1. ヒートマップ画像を保存
                overlaid_image = apply_heatmap_to_image(img, heatmap_resized)
                cv2.imwrite(str(heatmap_path), overlaid_image)

                # 2. AOIマスク画像を保存
                threshold_value = np.percentile(heatmap_resized, 50)
                mask = (heatmap_resized >= threshold_value).astype(np.uint8) * 255
                cv2.imwrite(str(mask_path), mask)

        except Exception as e:
            print(f"エラー発生: {basename}, Layer: {layer_name}, Error: {e}")

print("✅ GradCAM++のヒートマップとAOIマスクの生成が完了しました。")

2.2. GradCAMによる分析

基本的なGradCAMを6つの対象レイヤーに適用し、「元画像と重ね合わせたヒートマップ」と「AOIマスク」の両方を、それぞれ専用のフォルダに保存します。

In [ ]:
# --- メイン処理 ---
df = pd.read_csv(MAIN_CSV_PATH)

target_layers = [
    "model_17_cv3_conv", "model_20_cv3_conv", "model_23_cv3_conv",
    "model_24_m_0", "model_24_m_1", "model_24_m_2"
]

# === 拡張子対応: 事前に画像ファイルのパス辞書を作成 ===
VALID_EXTENSIONS = {'.jpg', '.jpeg', '.png'}
path_map = {
    Path(f).stem: Path(IMAGE_DIR) / f
    for f in os.listdir(IMAGE_DIR)
    if Path(f).suffix.lower() in VALID_EXTENSIONS
}
print(f"✅ {len(path_map)}件の画像ファイルをインデックスしました。")

# --- 出力ディレクトリの準備 ---
base_output_dir = Path(GRADCAM_MASK_DIR)
heatmap_base_dir = base_output_dir / "heatmaps"
mask_base_dir = base_output_dir / "masks"
for layer_name in target_layers:
    (heatmap_base_dir / layer_name).mkdir(parents=True, exist_ok=True)
    (mask_base_dir / layer_name).mkdir(parents=True, exist_ok=True)

print(f"GradCAM分析を開始します。保存先: {base_output_dir}")

# 画像ごとにループ
for index, row in tqdm(df.iterrows(), total=len(df), desc="GradCAM 生成中"):
    basename = row['image_basename']

    img_path_found = path_map.get(basename)
    if not img_path_found:
        continue

    # 1つのレイヤーでも処理済みか簡易チェック
    first_layer_mask_path = mask_base_dir / target_layers[0] / f"{basename}.png"
    if first_layer_mask_path.exists():
        continue

    img = cv2.imread(str(img_path_found))
    img_tensor, _ = preprocess_image(img, device=DEVICE)

    # 各レイヤーに対してループ
    for layer_name in target_layers:
        heatmap_path = heatmap_base_dir / layer_name / f"{basename}.jpg"
        mask_path = mask_base_dir / layer_name / f"{basename}.png"

        if heatmap_path.exists() and mask_path.exists():
            continue

        try:
            # methodを"gradcam"に指定して実行
            with YOLOV5GradCAM(model, layer_name, method="gradcam") as cam:
                heatmap = cam(img_tensor)

            if heatmap is not None:
                original_h, original_w = img.shape[:2]
                heatmap_resized = cv2.resize(heatmap, (original_w, original_h), interpolation=cv2.INTER_LINEAR)

                overlaid_image = apply_heatmap_to_image(img, heatmap_resized)
                cv2.imwrite(str(heatmap_path), overlaid_image)

                threshold_value = np.percentile(heatmap_resized, 50)
                mask = (heatmap_resized >= threshold_value).astype(np.uint8) * 255
                cv2.imwrite(str(mask_path), mask)

        except Exception as e:
            print(f"エラー発生: {basename}, Layer: {layer_name}, Error: {e}")

print("✅ GradCAMのヒートマップとAOIマスクの生成が完了しました。")

2.3. LIMEによる分析

各画像に対してLIMEを実行し、判断に寄与したスーパーピクセルを特定します。その結果を「元画像と重ね合わせたヒートマップ」と「評価用のAOIマスク」として保存します。

In [ ]:
# LIME分析に必要なクラスと関数
class SimpleLIME:
    """LIMEの簡易実装（バッチ処理対応）"""
    def __init__(self, model, device='cpu'):
        self.model = model
        self.device = device
        self.model.eval()

    def explain(self, img_bgr, target_class, num_samples=50):
        """LIME分析を実行"""
        h, w = img_bgr.shape[:2]

        # 1. スーパーピクセルに分割
        segments = quickshift(cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB), kernel_size=15, max_dist=200, ratio=0.2)
        num_segments = np.max(segments) + 1

        # 2. 摂動サンプルの生成と推論
        data = []
        labels = []

        # tqdmをループの外側に配置
        pbar = tqdm(range(num_samples), desc=" LIME Samples", leave=False)
        for _ in pbar:
            mask_vector = np.random.randint(0, 2, num_segments)
            masked_img = img_bgr.copy()
            for seg_id in range(num_segments):
                if mask_vector[seg_id] == 0:
                    masked_img[segments == seg_id] = 128 # グレーでマスク

            with torch.no_grad():
                img_tensor, _ = preprocess_image(masked_img, self.device)
                pred = self.model(img_tensor)[0]
                score = self._get_class_score(pred, target_class)

            data.append(mask_vector)
            labels.append(score)

        # 3. 線形モデルで学習
        from sklearn.linear_model import Ridge
        model_ridge = Ridge(alpha=1.0)
        model_ridge.fit(np.array(data), np.array(labels))
        importance = model_ridge.coef_

        # 4. 説明マップ（ヒートマップ）の作成
        explanation_map = np.zeros_like(segments, dtype=float)
        for seg_id in range(num_segments):
            explanation_map[segments == seg_id] = importance[seg_id]

        if np.max(explanation_map) > np.min(explanation_map):
            explanation_map = (explanation_map - np.min(explanation_map)) / (np.max(explanation_map) - np.min(explanation_map))

        return explanation_map

    def _get_class_score(self, prediction, target_class):
        """特定クラスの最高信頼度スコアを取得"""
        pred_nms = yolo_nms(prediction, conf_thres=0.01, iou_thres=IOU_THRES)[0]
        if pred_nms is None or len(pred_nms) == 0: return 0.0
        class_detections = pred_nms[pred_nms[:, 5] == target_class]
        return class_detections[:, 4].max().item() if len(class_detections) > 0 else 0.0

# --- メイン処理 ---
df = pd.read_csv(MAIN_CSV_PATH)

# 事前に画像ファイルのパス辞書を作成
VALID_EXTENSIONS = {'.jpg', '.jpeg', '.png'}
path_map = { Path(f).stem: Path(IMAGE_DIR) / f for f in os.listdir(IMAGE_DIR) if Path(f).suffix.lower() in VALID_EXTENSIONS }
print(f"✅ {len(path_map)}件の画像ファイルをインデックスしました。")

# 出力ディレクトリの準備
base_output_dir = Path(LIME_MASK_DIR)
heatmap_dir = base_output_dir / "heatmaps"
mask_dir = base_output_dir / "masks"
heatmap_dir.mkdir(parents=True, exist_ok=True)
mask_dir.mkdir(parents=True, exist_ok=True)

print(f"LIME分析を開始します。計算に時間がかかります...")
print(f"ヒートマップ保存先: {heatmap_dir}")
print(f"AOIマスク保存先: {mask_dir}")

for index, row in tqdm(df.iterrows(), total=len(df), desc="LIME 生成中"):
    basename = row['image_basename']
    heatmap_path = heatmap_dir / f"{basename}.jpg"
    mask_path = mask_dir / f"{basename}.png"

    if heatmap_path.exists() and mask_path.exists():
        continue

    img_path_found = path_map.get(basename)
    if not img_path_found:
        continue

    predicted_class_name = row['Predict']
    if pd.isna(predicted_class_name) or predicted_class_name == "none":
        continue
    target_class_idx = CLASS_NAMES.index(predicted_class_name)

    try:
        img_original = cv2.imread(str(img_path_found))
        img_resized = resize_image_with_aspect_ratio(img_original, target_width=640)

        lime_analyzer = SimpleLIME(model, device=DEVICE)
        lime_map = lime_analyzer.explain(img_resized, target_class_idx, num_samples=50)

        original_h, original_w = img_original.shape[:2]
        heatmap_resized = cv2.resize(lime_map, (original_w, original_h), interpolation=cv2.INTER_LINEAR)

        overlaid_image = apply_heatmap_to_image(img_original, heatmap_resized)
        cv2.imwrite(str(heatmap_path), overlaid_image)

        threshold_value = np.percentile(heatmap_resized, 50)
        mask = (heatmap_resized >= threshold_value).astype(np.uint8) * 255
        cv2.imwrite(str(mask_path), mask)

    except Exception as e:
        print(f"エラー発生: {basename}, Error: {e}")

print(f"✅ LIMEのヒートマップとAOIマスクの生成が完了しました。")

2.4. Occlusion Sensitivityによる分析

各画像に対してOcclusion Sensitivity分析を実行し、その結果を「元画像と重ね合わせたヒートマップ」と「評価用のAOIマスク」として、それぞれ専用のフォルダに保存します。

In [ ]:
# Occlusion Sensitivity分析に必要なクラスと関数
class OcclusionSensitivity:
    def __init__(self, model, device='cpu'):
        self.model = model
        self.device = device
        self.model.eval()

    def analyze(self, img_bgr, target_class, bbox, patch_ratio=0.15, stride_ratio=0.2, batch_size=32):
        h, w = img_bgr.shape[:2]

        # パッチサイズとストライドを計算
        bbox_width = bbox[2] - bbox[0]
        bbox_height = bbox[3] - bbox[1]
        patch_size = int(min(bbox_width, bbox_height) * patch_ratio)
        patch_size = max(20, min(patch_size, 100))
        stride = max(1, int(patch_size * stride_ratio))

        # 元画像での予測スコアを取得
        with torch.no_grad():
            img_tensor, _ = preprocess_image(img_bgr, self.device)
            base_pred = self.model(img_tensor)[0]
            base_score = self._get_class_score(base_pred, target_class)

        # 遮蔽する位置のリストを作成
        positions = []
        for y in range(0, h - patch_size + 1, stride):
            for x in range(0, w - patch_size + 1, stride):
                positions.append((y, x))

        sensitivity_map = np.zeros((h, w), dtype=np.float32)
        counts_map = np.zeros((h, w), dtype=np.float32)

        for i in tqdm(range(0, len(positions), batch_size), desc=" Occlusion Batches", leave=False):
            batch_positions = positions[i:i + batch_size]
            batch_imgs = [img_bgr.copy() for _ in batch_positions]

            for idx, (y, x) in enumerate(batch_positions):
                batch_imgs[idx][y:y+patch_size, x:x+patch_size] = 128 # グレーで遮蔽

            with torch.no_grad():
                batch_tensor = torch.cat([preprocess_image(img, self.device)[0] for img in batch_imgs])
                batch_pred = self.model(batch_tensor)[0]

            for idx, (y, x) in enumerate(batch_positions):
                score = self._get_class_score(batch_pred[idx:idx+1], target_class)
                score_drop = base_score - score
                sensitivity_map[y:y+patch_size, x:x+patch_size] += score_drop
                counts_map[y:y+patch_size, x:x+patch_size] += 1

        # 重複領域を平均化し正規化
        sensitivity_map = np.divide(sensitivity_map, counts_map, out=np.zeros_like(sensitivity_map), where=counts_map!=0)
        if np.max(sensitivity_map) > np.min(sensitivity_map):
            sensitivity_map = (sensitivity_map - np.min(sensitivity_map)) / (np.max(sensitivity_map) - np.min(sensitivity_map))

        return sensitivity_map

    def _get_class_score(self, prediction, target_class):
        pred_nms = yolo_nms(prediction, conf_thres=0.01, iou_thres=IOU_THRES)[0]
        if pred_nms is None or len(pred_nms) == 0: return 0.0
        class_detections = pred_nms[pred_nms[:, 5] == target_class]
        return class_detections[:, 4].max().item() if len(class_detections) > 0 else 0.0

# --- メイン処理 ---
df = pd.read_csv(MAIN_CSV_PATH)
path_map = { Path(f).stem: Path(IMAGE_DIR) / f for f in os.listdir(IMAGE_DIR) if Path(f).suffix.lower() in {'.jpg', '.jpeg', '.png'} }
print(f"✅ {len(path_map)}件の画像ファイルをインデックスしました。")

# 出力ディレクトリの準備
base_output_dir = Path(OCCLUSION_MASK_DIR)
heatmap_dir = base_output_dir / "heatmaps"
mask_dir = base_output_dir / "masks"
heatmap_dir.mkdir(parents=True, exist_ok=True)
mask_dir.mkdir(parents=True, exist_ok=True)

print(f"Occlusion Sensitivity分析を開始します...")
print(f"ヒートマップ保存先: {heatmap_dir}")
print(f"AOIマスク保存先: {mask_dir}")

for index, row in tqdm(df.iterrows(), total=len(df), desc="Occlusion Sensitivity 生成中"):
    basename = row['image_basename']
    heatmap_path = heatmap_dir / f"{basename}.jpg"
    mask_path = mask_dir / f"{basename}.png"

    if heatmap_path.exists() and mask_path.exists():
        continue

    img_path_found = path_map.get(basename)
    if not img_path_found:
        continue

    predicted_class_name = row['Predict']
    if pd.isna(predicted_class_name) or predicted_class_name == "none":
        continue
    target_class_idx = CLASS_NAMES.index(predicted_class_name)

    try:
        img_original = cv2.imread(str(img_path_found))
        img_resized = resize_image_with_aspect_ratio(img_original, target_width=640)

        # BBoxを再計算
        with torch.no_grad():
            pred_tensor, _ = preprocess_image(img_resized, device=DEVICE)
            pred = model(pred_tensor)[0]
            pred_nms = yolo_nms(pred, conf_thres=CONF_THRES, iou_thres=IOU_THRES)[0]
            if pred_nms is None or len(pred_nms) == 0: continue
            bbox = pred_nms[0, :4].cpu().numpy()

        occlusion_analyzer = OcclusionSensitivity(model, device=DEVICE)
        heatmap = occlusion_analyzer.analyze(img_resized, target_class_idx, bbox)

        # 元の画像サイズにリサイズ
        original_h, original_w = img_original.shape[:2]
        heatmap_resized = cv2.resize(heatmap, (original_w, original_h), interpolation=cv2.INTER_LINEAR)

        # 1. ヒートマップ画像を保存
        overlaid_image = apply_heatmap_to_image(img_original, heatmap_resized)
        cv2.imwrite(str(heatmap_path), overlaid_image)

        # 2. AOIマスク画像を保存
        threshold_value = np.percentile(heatmap_resized, 50)
        mask = (heatmap_resized >= threshold_value).astype(np.uint8) * 255
        cv2.imwrite(str(mask_path), mask)

    except Exception as e:
        print(f"エラー発生: {basename}, Error: {e}")

print(f"✅ Occlusion SensitivityのヒートマップとAOIマスクの生成が完了しました。")

2.5. RISEによる分析 (比較実験用)

In [ ]:
# --- RISE分析に必要なクラスと関数 ---

def non_max_suppression_multilabel(prediction, conf_thres=0.25, iou_thres=0.45, max_det=300):
    """マルチラベル対応のNMS。各検出に対して複数クラスの確率を保持します。"""
    nc = prediction.shape[2] - 5
    xc = prediction[..., 4] > conf_thres

    output = [torch.zeros((0, 6 + nc), device=prediction.device)] * prediction.shape[0]
    for xi, x in enumerate(prediction):
        x = x[xc[xi]]
        if not x.shape[0]:
            continue

        box = xywh2xyxy(x[:, :4])
        x[:, 5:] *= x[:, 4:5]

        conf, j = x[:, 5:].max(1, keepdim=True)
        x_nms = torch.cat((box, conf, j.float()), 1)

        # 信頼度でフィルタリング
        x_nms = x_nms[conf.view(-1) > conf_thres]
        x = x[conf.view(-1) > conf_thres]
        if not x_nms.shape[0]:
            continue

        c = x_nms[:, 5:6] * 4096
        boxes, scores = x_nms[:, :4] + c, x_nms[:, 4]
        i = torchvision.ops.nms(boxes, scores, iou_thres)
        if i.shape[0] > max_det: i = i[:max_det]

        # NMSで選択されたインデックスに対応する元の全クラス情報を保持
        output[xi] = torch.cat((x_nms[i, :6], x[i, 5:]), 1)

    return output

class YOLOV5TorchObjectDetectorML(nn.Module):
    """RISEのために全クラスの確率を出力するYOLOv5ラッパー"""
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, img):
        prediction, _, _ = self.model(img, augment=False)
        return non_max_suppression_multilabel(prediction, conf_thres=0.01, iou_thres=IOU_THRES)

class RISE:
    def __init__(self, model, input_size=(640, 640), n_masks=1000, p1=0.5, s=16):
        self.wrapped_model = YOLOV5TorchObjectDetectorML(model)
        self.input_size = input_size
        self.n_masks = n_masks
        self.p1 = p1
        self.s = s
        self.masks = self._generate_masks().to(model.device)

    def _generate_masks(self):
        h = (self.input_size[0] + self.s - 1) // self.s
        w = (self.input_size[1] + self.s - 1) // self.s
        masks_np = (np.random.rand(self.n_masks, h, w) < self.p1).astype(np.float32)
        return torch.from_numpy(masks_np)

    def explain(self, img_tensor, target_class_idx, batch_size=50):
        saliency = torch.zeros((1, 1, *self.input_size)).to(img_tensor.device)

        for i in range(0, self.n_masks, batch_size):
            masks_batch = self.masks[i:i + batch_size]
            masks_resized = F.interpolate(masks_batch.unsqueeze(1), size=self.input_size, mode='bilinear', align_corners=False)

            masked_imgs = img_tensor * masks_resized

            with torch.no_grad():
                preds = self.wrapped_model(masked_imgs)

            for j, p in enumerate(preds):
                score = 0.0
                if p is not None and len(p) > 0:
                    target_probs = p[:, 6 + target_class_idx]
                    if len(target_probs) > 0:
                        score = target_probs.max().item()
                saliency += masks_resized[j] * score

        saliency /= self.n_masks
        saliency = (saliency - saliency.min()) / (saliency.max() - saliency.min() + 1e-8)
        return saliency.squeeze().cpu().numpy()

# --- メイン処理 ---
df = pd.read_csv(MAIN_CSV_PATH)
path_map = { Path(f).stem: Path(IMAGE_DIR) / f for f in os.listdir(IMAGE_DIR) if Path(f).suffix.lower() in {'.jpg', '.jpeg', '.png'} }

# RISE用の出力ディレクトリを定義
RISE_MASK_DIR = os.path.join(BASE_DIR, "AOI_50_mask_RISE")
base_output_dir = Path(RISE_MASK_DIR)
heatmap_dir = base_output_dir / "heatmaps"
mask_dir = base_output_dir / "masks"
heatmap_dir.mkdir(parents=True, exist_ok=True)
mask_dir.mkdir(parents=True, exist_ok=True)

print("RISE分析を開始します。")
print(f"ヒートマップ保存先: {heatmap_dir}")
print(f"AOIマスク保存先: {mask_dir}")

for index, row in tqdm(df.iterrows(), total=len(df), desc="RISE 生成中"):
    basename = row['image_basename']
    heatmap_path = heatmap_dir / f"{basename}.jpg"
    mask_path = mask_dir / f"{basename}.png"

    if heatmap_path.exists() and mask_path.exists():
        continue

    img_path_found = path_map.get(basename)
    if not img_path_found:
        continue

    gt_class_name = row['GroundTruth']
    if pd.isna(gt_class_name):
        continue
    target_class_idx = CLASS_NAMES.index(gt_class_name)

    try:
        img_original = cv2.imread(str(img_path_found))
        img_tensor, _ = preprocess_image(img_original, device=DEVICE)

        rise_analyzer = RISE(model, n_masks=1000)
        heatmap = rise_analyzer.explain(img_tensor, target_class_idx)

        original_h, original_w = img_original.shape[:2]
        heatmap_resized = cv2.resize(heatmap, (original_w, original_h), interpolation=cv2.INTER_LINEAR)

        overlaid_image = apply_heatmap_to_image(img_original, heatmap_resized)
        cv2.imwrite(str(heatmap_path), overlaid_image)

        threshold_value = np.percentile(heatmap_resized, 50)
        mask = (heatmap_resized >= threshold_value).astype(np.uint8) * 255
        cv2.imwrite(str(mask_path), mask)

    except Exception as e:
        print(f"エラー発生: {basename}, Error: {e}")

print("✅ RISEのヒートマップとAOIマスクの生成が完了しました。")

##第3部：評価指標の計算と記録


3.0. 評価用共通関数の定義

まず、Pointing GameとIoUの計算で共通して使用するヘルパー関数を定義します。

In [ ]:
def get_max_point_from_heatmap(heatmap_path):
    """保存されたヒートマップ画像から最大値の座標(x, y)を取得する"""
    heatmap_img = cv2.imread(str(heatmap_path), cv2.IMREAD_GRAYSCALE)
    if heatmap_img is None:
        return None, None

    max_idx = np.argmax(heatmap_img)
    max_y, max_x = np.unravel_index(max_idx, heatmap_img.shape)

    return int(max_x), int(max_y)

def calculate_iou(mask1_path, mask2_path):
    """2つのマスク画像のパスからIoUを計算する"""
    mask1 = cv2.imread(str(mask1_path), cv2.IMREAD_GRAYSCALE)
    mask2 = cv2.imread(str(mask2_path), cv2.IMREAD_GRAYSCALE)

    if mask1 is None or mask2 is None:
        return np.nan

    # 異なるサイズのマスクはリサイズして合わせる
    if mask1.shape != mask2.shape:
        mask2 = cv2.resize(mask2, (mask1.shape[1], mask1.shape[0]), interpolation=cv2.INTER_NEAREST)

    mask1_bool = mask1 > 128 # 閾値処理
    mask2_bool = mask2 > 128 # 閾値処理

    intersection = np.logical_and(mask1_bool, mask2_bool).sum()
    union = np.logical_or(mask1_bool, mask2_bool).sum()

    return intersection / union if union > 0 else 0.0

print("✅ 評価用共通関数の定義完了")

3.1. GradCAM++の評価指標計算

model_23_cv3_convレイヤーの結果を代表として、Pointing GameとIoUを計算し、結果をCSVに保存します。

In [ ]:
# --- メイン処理 ---
df = pd.read_csv(MAIN_CSV_PATH)
target_layer = "model_23_cv3_conv"

heatmap_dir = Path(GRADCAM_PP_MASK_DIR) / "heatmaps" / target_layer
mask_dir = Path(GRADCAM_PP_MASK_DIR) / "masks" / target_layer
expert_dir = Path(EXPERT_MASK_DIR)

print(f"GradCAM++ ({target_layer}) の評価を開始します...")

for index, row in tqdm(df.iterrows(), total=len(df), desc="GradCAM++ 評価中"):
    # resume機能: 既に評価済みの場合はスキップ
    if pd.notna(row.get('GradCAM++_IoU')):
        continue

    basename = row['image_basename']

    # 必要なファイルのパスを定義
    heatmap_path = heatmap_dir / f"{basename}.jpg"
    aoi_mask_path = mask_dir / f"{basename}.png"
    expert_mask_path = expert_dir / f"{basename}.png"

    # 専門家マスクと、評価対象のファイルが存在するかチェック
    if not expert_mask_path.exists() or not heatmap_path.exists() or not aoi_mask_path.exists():
        continue

    # 1. Pointing Game
    max_x, max_y = get_max_point_from_heatmap(heatmap_path)
    if max_x is not None:
        expert_mask_img = cv2.imread(str(expert_mask_path), cv2.IMREAD_GRAYSCALE)
        is_hit = 1 if expert_mask_img[max_y, max_x] > 0 else 0

        df.loc[index, 'GradCAM++_X'] = max_x
        df.loc[index, 'GradCAM++_Y'] = max_y
        df.loc[index, 'GradCAM++_pointing_game'] = is_hit

    # 2. IoU
    iou = calculate_iou(aoi_mask_path, expert_mask_path)
    df.loc[index, 'GradCAM++_IoU'] = iou

# --- 結果を保存 ---
df.to_csv(MAIN_CSV_PATH, index=False, encoding='utf-8-sig')
print(f"✅ GradCAM++の評価結果をCSVに保存しました。")
display(df[['image_basename', 'GradCAM++_pointing_game', 'GradCAM++_IoU']].dropna().head())

3.2. GradCAMの評価指標計算

model_23_cv3_convレイヤーの結果を代表として、Pointing GameとIoUを計算し、結果をCSVに保存します。

In [ ]:
# --- メイン処理 ---
df = pd.read_csv(MAIN_CSV_PATH)
target_layer = "model_23_cv3_conv"

heatmap_dir = Path(GRADCAM_MASK_DIR) / "heatmaps" / target_layer
mask_dir = Path(GRADCAM_MASK_DIR) / "masks" / target_layer
expert_dir = Path(EXPERT_MASK_DIR)

print(f"GradCAM ({target_layer}) の評価を開始します...")

for index, row in tqdm(df.iterrows(), total=len(df), desc="GradCAM 評価中"):
    # resume機能: 既に評価済みの場合はスキップ
    if pd.notna(row.get('GradCAM_IoU')):
        continue

    basename = row['image_basename']

    # 必要なファイルのパスを定義
    heatmap_path = heatmap_dir / f"{basename}.jpg"
    aoi_mask_path = mask_dir / f"{basename}.png"
    expert_mask_path = expert_dir / f"{basename}.png"

    # 専門家マスクと、評価対象のファイルが存在するかチェック
    if not expert_mask_path.exists() or not heatmap_path.exists() or not aoi_mask_path.exists():
        continue

    # 1. Pointing Game
    max_x, max_y = get_max_point_from_heatmap(heatmap_path)
    if max_x is not None:
        expert_mask_img = cv2.imread(str(expert_mask_path), cv2.IMREAD_GRAYSCALE)
        is_hit = 1 if expert_mask_img[max_y, max_x] > 0 else 0

        df.loc[index, 'GradCAM_X'] = max_x
        df.loc[index, 'GradCAM_Y'] = max_y
        df.loc[index, 'GradCAM_pointing_game'] = is_hit

    # 2. IoU
    iou = calculate_iou(aoi_mask_path, expert_mask_path)
    df.loc[index, 'GradCAM_IoU'] = iou

# --- 結果を保存 ---
df.to_csv(MAIN_CSV_PATH, index=False, encoding='utf-8-sig')
print(f"✅ GradCAMの評価結果をCSVに保存しました。")
display(df[['image_basename', 'GradCAM_pointing_game', 'GradCAM_IoU']].dropna().head())

3.3. LIMEの評価指標計算

AOI_50_mask_LIMEフォルダに保存されたヒートマップとマスクを使用し、「Pointing Game」と「IoU」を計算してCSVに保存します。

In [ ]:
# --- メイン処理 ---
df = pd.read_csv(MAIN_CSV_PATH)

heatmap_dir = Path(LIME_MASK_DIR) / "heatmaps"
mask_dir = Path(LIME_MASK_DIR) / "masks"
expert_dir = Path(EXPERT_MASK_DIR)

print(f"LIME の評価を開始します...")

for index, row in tqdm(df.iterrows(), total=len(df), desc="LIME 評価中"):
    # resume機能: 既に評価済みの場合はスキップ
    if pd.notna(row.get('LIME_IoU')):
        continue

    basename = row['image_basename']

    # 必要なファイルのパスを定義
    heatmap_path = heatmap_dir / f"{basename}.jpg"
    aoi_mask_path = mask_dir / f"{basename}.png"
    expert_mask_path = expert_dir / f"{basename}.png"

    # 専門家マスクと、評価対象のファイルが存在するかチェック
    if not expert_mask_path.exists() or not heatmap_path.exists() or not aoi_mask_path.exists():
        continue

    # 1. Pointing Game
    max_x, max_y = get_max_point_from_heatmap(heatmap_path)
    if max_x is not None:
        expert_mask_img = cv2.imread(str(expert_mask_path), cv2.IMREAD_GRAYSCALE)
        # 座標がマスクの範囲内にあるか確認
        if max_y < expert_mask_img.shape[0] and max_x < expert_mask_img.shape[1]:
            is_hit = 1 if expert_mask_img[max_y, max_x] > 0 else 0
        else:
            is_hit = 0 # 座標が範囲外の場合はヒットしない

        df.loc[index, 'LIME_X'] = max_x
        df.loc[index, 'LIME_Y'] = max_y
        df.loc[index, 'LIME_pointing_game'] = is_hit

    # 2. IoU
    iou = calculate_iou(aoi_mask_path, expert_mask_path)
    df.loc[index, 'LIME_IoU'] = iou

# --- 結果を保存 ---
df.to_csv(MAIN_CSV_PATH, index=False, encoding='utf-8-sig')
print(f"✅ LIMEの評価結果をCSVに保存しました。")
display(df[['image_basename', 'LIME_pointing_game', 'LIME_IoU']].dropna().head())

3.4. Occlusion Sensitivityの評価指標計算

AOI_50_mask_Occlusion_sensitivityフォルダに保存されたヒートマップとマスクを使用し、「Pointing Game」と「IoU」を計算してCSVに保存します。

In [ ]:
# --- メイン処理 ---
df = pd.read_csv(MAIN_CSV_PATH)

heatmap_dir = Path(OCCLUSION_MASK_DIR) / "heatmaps"
mask_dir = Path(OCCLUSION_MASK_DIR) / "masks"
expert_dir = Path(EXPERT_MASK_DIR)

print(f"Occlusion Sensitivity の評価を開始します...")

for index, row in tqdm(df.iterrows(), total=len(df), desc="Occlusion Sensitivity 評価中"):
    # resume機能: 既に評価済みの場合はスキップ
    if pd.notna(row.get('Occlusion_IoU')):
        continue

    basename = row['image_basename']

    # 必要なファイルのパスを定義
    heatmap_path = heatmap_dir / f"{basename}.jpg"
    aoi_mask_path = mask_dir / f"{basename}.png"
    expert_mask_path = expert_dir / f"{basename}.png"

    # 専門家マスクと、評価対象のファイルが存在するかチェック
    if not expert_mask_path.exists() or not heatmap_path.exists() or not aoi_mask_path.exists():
        continue

    # 1. Pointing Game
    max_x, max_y = get_max_point_from_heatmap(heatmap_path)
    if max_x is not None:
        expert_mask_img = cv2.imread(str(expert_mask_path), cv2.IMREAD_GRAYSCALE)
        # 座標がマスクの範囲内にあるか確認
        if max_y < expert_mask_img.shape[0] and max_x < expert_mask_img.shape[1]:
            is_hit = 1 if expert_mask_img[max_y, max_x] > 0 else 0
        else:
            is_hit = 0 # 座標が範囲外の場合はヒットしない

        df.loc[index, 'Occlusion_X'] = max_x
        df.loc[index, 'Occlusion_Y'] = max_y
        df.loc[index, 'Occlusion_pointing_game'] = is_hit

    # 2. IoU
    iou = calculate_iou(aoi_mask_path, expert_mask_path)
    df.loc[index, 'Occlusion_IoU'] = iou

# --- 結果を保存 ---
df.to_csv(MAIN_CSV_PATH, index=False, encoding='utf-8-sig')
print(f"✅ Occlusion Sensitivityの評価結果をCSVに保存しました。")
display(df[['image_basename', 'Occlusion_pointing_game', 'Occlusion_IoU']].dropna().head())

3.5. RISEの評価指標計算

AOI_50_mask_RISEフォルダに保存されたヒートマップとマスクを使用し、「Pointing Game」と「IoU」を計算してCSVに保存します。

In [ ]:
# --- メイン処理 ---
df = pd.read_csv(MAIN_CSV_PATH)

heatmap_dir = Path(RISE_MASK_DIR) / "heatmaps"
mask_dir = Path(RISE_MASK_DIR) / "masks"
expert_dir = Path(EXPERT_MASK_DIR)

print(f"RISE の評価を開始します...")

for index, row in tqdm(df.iterrows(), total=len(df), desc="RISE 評価中"):
    # resume機能: 既に評価済みの場合はスキップ
    if pd.notna(row.get('RISE_IoU')):
        continue

    basename = row['image_basename']

    # 必要なファイルのパスを定義
    heatmap_path = heatmap_dir / f"{basename}.jpg"
    aoi_mask_path = mask_dir / f"{basename}.png"
    expert_mask_path = expert_dir / f"{basename}.png"

    # 専門家マスクと、評価対象のファイルが存在するかチェック
    if not expert_mask_path.exists() or not heatmap_path.exists() or not aoi_mask_path.exists():
        continue

    # 1. Pointing Game
    max_x, max_y = get_max_point_from_heatmap(heatmap_path)
    if max_x is not None:
        expert_mask_img = cv2.imread(str(expert_mask_path), cv2.IMREAD_GRAYSCALE)
        # 座標がマスクの範囲内にあるか確認
        if max_y < expert_mask_img.shape[0] and max_x < expert_mask_img.shape[1]:
            is_hit = 1 if expert_mask_img[max_y, max_x] > 0 else 0
        else:
            is_hit = 0 # 座標が範囲外の場合はヒットしない

        df.loc[index, 'RISE_X'] = max_x
        df.loc[index, 'RISE_Y'] = max_y
        df.loc[index, 'RISE_pointing_game'] = is_hit

    # 2. IoU
    iou = calculate_iou(aoi_mask_path, expert_mask_path)
    df.loc[index, 'RISE_IoU'] = iou

# --- 結果を保存 ---
df.to_csv(MAIN_CSV_PATH, index=False, encoding='utf-8-sig')
print(f"✅ RISEの評価結果をCSVに保存しました。")
display(df[['image_basename', 'RISE_pointing_game', 'RISE_IoU']].dropna().head())

##第4部：Cut & Paste実験

角膜領域を別の画像に移植し、モデルが背景の変化に対してどれだけ頑健であるかを評価します。

4.1. 実験用CSVの作成 (Ueno_Mix1039_cutmix.csv)

まず、移植元（cornea）と移植先（background）の画像の組み合わせを網羅したCSVファイルを作成します。ここでは、元論文のコードにあったUeno_Mix1039_over90.csvという高画質な画像リストを基に作成するロジックを再現します。

【注意】 この処理を実行する前に、Ueno_Mix1039_over90.csvがBASE_DIRに存在するかご確認ください。

In [ ]:
# --- 4.1. 実験ペアリストの作成 ---

# 高画質リストとされるCSVを読み込む
try:
    source_csv_path = os.path.join(BASE_DIR, "Ueno_Mix1039_over90.csv")
    df_quality = pd.read_csv(source_csv_path)
except FileNotFoundError:
    print(f"エラー: {source_csv_path} が見つかりません。このセルはスキップされます。")
    df_quality = None

if df_quality is not None:
    # 画質が良いものだけを抽出
    df_high_quality = df_quality[df_quality["Image_quality"] == 1].copy()

    # 背景用と角膜用のDataFrameを準備
    df_bg = df_high_quality[['image_basename']].rename(columns={"image_basename": "background_basename"})
    df_cornea = df_high_quality[['image_basename']].rename(columns={"image_basename": "cornea_basename"})

    # 全ての組み合わせ（デカルト積）を作成
    df_bg['key'] = 1
    df_cornea['key'] = 1
    cutmix_df = pd.merge(df_bg, df_cornea, on='key').drop('key', axis=1)

    # 予測結果を格納する空の列を追加
    cutmix_df['cornea_pred'] = pd.NA
    cutmix_df['cutmix_pred'] = pd.NA

    # CSVとして保存
    cutmix_df.to_csv(CUTMIX_CSV_PATH, index=False, encoding='utf-8-sig')

    print(f"✅ Cut & Paste実験用のペアリストを作成しました。")
    print(f"   - 保存先: {CUTMIX_CSV_PATH}")
    print(f"   - 総ペア数: {len(cutmix_df)}")
    display(cutmix_df.head())

4.2. & 4.3. Cut & Pasteの実行と推論

4.1で作成したペアリストに基づき、実際に角膜領域の切り貼りを行い、合成画像に対してAIの推論を実行します。処理に時間がかかるため、100件ごとに中間保存を行います。

In [ ]:
# --- 4.2 & 4.3. 画像合成、推論、結果の記録 ---

# --- Helper Functions ---
def get_ellipse_from_xml(xml_path, image_name):
    """XMLファイルから楕円情報を取得"""
    # 毎回ファイルをパースするのは非効率なため、初回のみパース結果をキャッシュする
    if not hasattr(get_ellipse_from_xml, 'root'):
        try:
            tree = ET.parse(xml_path)
            get_ellipse_from_xml.root = tree.getroot()
        except FileNotFoundError:
            get_ellipse_from_xml.root = None
            return None

    if get_ellipse_from_xml.root is None:
        return None

    for image_node in get_ellipse_from_xml.root.findall('.//image'):
        if image_node.get('name') == image_name:
            ellipse_node = image_node.find('ellipse')
            if ellipse_node is not None:
                return {k: float(v) for k, v in ellipse_node.attrib.items()}
    return None

def affine_transplant(src_img, tgt_img, src_ellipse, tgt_ellipse):
    """アフィン変換を用いて角膜を移植する"""
    def get_transform_points(e):
        angle_rad = np.deg2rad(e.get('rotation', 0))
        cos_a, sin_a = np.cos(angle_rad), np.sin(angle_rad)
        cx, cy, rx, ry = e['cx'], e['cy'], e['rx'], e['ry']
        return np.float32([
            [cx, cy],
            [cx + rx * cos_a, cy + rx * sin_a],
            [cx - ry * sin_a, cy + ry * cos_a]
        ])

    M = cv2.getAffineTransform(get_transform_points(src_ellipse), get_transform_points(tgt_ellipse))
    warped = cv2.warpAffine(src_img, M, (tgt_img.shape[1], tgt_img.shape[0]))

    mask = np.zeros(tgt_img.shape[:2], dtype=np.uint8)
    cv2.ellipse(mask,
                (int(tgt_ellipse['cx']), int(tgt_ellipse['cy'])),
                (int(tgt_ellipse['rx']), int(tgt_ellipse['ry'])),
                tgt_ellipse.get('rotation', 0), 0, 360, 255, -1)

    # 境界をぼかして自然に合成
    mask_blurred = cv2.GaussianBlur(mask, (21, 21), 10)
    mask_3ch = cv2.cvtColor(mask_blurred, cv2.COLOR_GRAY2BGR).astype('float32') / 255.0

    return ((mask_3ch * warped) + ((1 - mask_3ch) * tgt_img)).astype(np.uint8)

# --- Main Execution Loop ---
try:
    df_cutmix = pd.read_csv(CUTMIX_CSV_PATH)

    # 未処理の行のみを対象にする
    to_process = df_cutmix[df_cutmix['cutmix_pred'].isna()]
    print(f"Cut & Paste推論を開始します。対象ペア数: {len(to_process)} / {len(df_cutmix)}")

    for index, row in tqdm(to_process.iterrows(), total=len(to_process), desc="Cut & Paste 推論中"):
        cornea_path = path_map.get(row['cornea_basename'])
        bg_path = path_map.get(row['background_basename'])

        if not cornea_path or not bg_path:
            continue

        cornea_ellipse = get_ellipse_from_xml(XML_ANNOTATION_PATH, cornea_path.name)
        bg_ellipse = get_ellipse_from_xml(XML_ANNOTATION_PATH, bg_path.name)

        if not cornea_ellipse or not bg_ellipse:
            df_cutmix.loc[index, 'cutmix_pred'] = 'no_ellipse'
            continue

        try:
            # 合成画像を作成して推論
            src_img = cv2.imread(str(cornea_path))
            tgt_img = cv2.imread(str(bg_path))
            cutmix_img = affine_transplant(src_img, tgt_img, cornea_ellipse, bg_ellipse)

            with torch.no_grad():
                img_tensor, _ = preprocess_image(cutmix_img, device=DEVICE)
                pred = model(img_tensor)[0]
                pred_nms = yolo_nms(pred, conf_thres=CONF_THRES, iou_thres=IOU_THRES)[0]

                if pred_nms is not None and len(pred_nms) > 0:
                    pred_class_idx = int(pred_nms[0][5])
                    df_cutmix.loc[index, 'cutmix_pred'] = CLASS_NAMES[pred_class_idx]
                else:
                    df_cutmix.loc[index, 'cutmix_pred'] = 'none'

            # 100件ごとに中間保存
            if (index + 1) % 100 == 0:
                df_cutmix.to_csv(CUTMIX_CSV_PATH, index=False, encoding='utf-8-sig')

        except Exception as e:
            print(f"エラー発生: {row['cornea_basename']} -> {row['background_basename']}, Error: {e}")
            df_cutmix.loc[index, 'cutmix_pred'] = 'error'

    # 最終保存
    df_cutmix.to_csv(CUTMIX_CSV_PATH, index=False, encoding='utf-8-sig')
    print(f"✅ Cut & Paste実験の推論が完了しました。")

except FileNotFoundError:
    print("Cut & Paste実験用CSVが存在しないため、このセルはスキップされました。")

##第5部：追加分析とデータ整理

5.1. AI検出AOI_50と専門家マスクの一致率計算 (expert_ratio)

AIが物体として認識した領域（バウンディングボックス）が、専門家が重要だと判断した領域（エキスパートマスク）とどの程度重なっているかを評価します。この指標は、AIが「どこを見ているか」と専門家の判断が一致しているかを示します。